# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [65]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import re

from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [69]:
stopwords = stopwords.words("english")

In [6]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [ ]:
convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT party, text FROM conventions
                            where party != "Other"
                            ''')

for row in query_results :
    # store the results in convention_data
    # remove this and replace with your code
    party = row[0]
    speech = row[1]
    convention_data.append([speech, party])


In [40]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right. 

In [39]:
random.choices(convention_data,k=5)

[['Hi, I’m Susan Molinari. I’m a former Republican member of Congress from New York City, and I’ve known Donald Trump for most of my political career. So disappointing and lately, so disturbing. Now, I’ve also gotten to know and work with Joe Biden on issues related to women that are so important to all of us, women in business violence against women. That’s why I’m so proud to call him my friend, and honored to join and supporting his candidacy for president. He’s a really good man and he’s exactly what this nation needs at this time. Now I’m delighted and honored to introduce a former colleague of mine, the former Congressman from Ohio, the former Governor of Ohio, John Kasich.',
  'Democratic'],
 ['This time next year.', 'Democratic'],
 ['Joe’s working to protect and expand the Affordable Care Act to make sure millions of people keep their coverage and no one can be denied for a pre-existing condition. To bring down the costs of healthcare and prescription drugs too, giving tax cred

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html). 

In [58]:
conv_sent_data = []

for speech, party in convention_data :
    first = nltk.sent_tokenize(speech)[0]
    conv_sent_data.append([first, party])


Again, let's look at some random entries. 

In [59]:
random.choices(conv_sent_data,k=5)

[['And then there’s Pastor Aaron Johnson.', 'Republican'],
 ['That I absolutely and entirely.', 'Republican'],
 ['President Trump will rebuild America’s infrastructure, renew hope, and our entrepreneurial spirit for a new generation, and restore the American dream.',
  'Republican'],
 ['He will make it his job to know when anyone dares to threaten us, he will stand up to our adversaries with strength and experience.',
  'Democratic'],
 ['My name is Abby Johnson, and I spent eight years working for Planned Parenthood, but today I’m a pro-life activist.',
  'Republican']]

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps: 

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [71]:
clean_conv_sent_data = [] # list of tuples (sentence, party), with sentence cleaned

for idx, sent_party in conv_sent_data :
    tokens = word_tokenize(idx)
    tokens = [token for token in tokens if token not in punctuation]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token.lower() not in stopwords]
    tokens = [token.lower() for token in tokens]
    sent = ' '.join(tokens)
    clean_conv_sent_data.append([sent, sent_party])
    
    

random.choices(clean_conv_sent_data,k=5)

[['han grew poverty immigrants able make decent living', 'Democratic'],
 ['something wrong', 'Democratic'],
 ['seen tragic shootings sides officers killing citizens citizens killing officers line duty',
  'Republican'],
 ['human cost devastating', 'Republican'],
 ['give children children children planet sustain', 'Democratic']]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [75]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 628 as features in the model.


In [76]:
def conv_features(text,feature_words) :
     ret_dict = dict()
     word = text.split()
     ret_dict = {word: True for word in word if word in feature_words}
     return(ret_dict)

In [77]:
assert(len(feature_words)>0)
assert(conv_features("obama was the president",feature_words)==
       {'obama':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [127]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [79]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [ ]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.494


In [81]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.3 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
              appreciate = True           Republ : Democr =     14.0 : 1.0
                    drug = True           Republ : Democr =     10.3 : 1.0
                 special = True           Republ : Democr =     10.3 : 1.0
                   local = True           Republ : Democr =      9.9 : 1.0
                   elect = True           Democr : Republ =      9.6 : 1.0
                    land = True           Republ : Democr =      8.9 : 1.0
                  cities = True           Republ : Democr =      8.4 : 1.0
                citizens = True           Republ : Democr =      8.4 : 1.0
                    flag = True           Republ : Democr =      8.4 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

It seems that the words "encorcement", "media", "appreciate", "drug", "special" and "local" are the words that indicate to this model that a speech is done by a Republican. The words "votes", "climate", "elect", and "clean" are the words that the model finds most important for classifying a speech is done by a Democrat. There seem to be a lot more words that indicate a Republican speech over a Democrat speech than vice versa. 



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [83]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [84]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [147]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results :
    # store the results in convention_data
    # remove this and replace with your code
    candidate = row[0]
    party = row[1]
    tweet = row[2]
    tweet_data.append([tweet, party])


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [212]:
random.seed(20250602)

tweet_data_sample = random.choices(tweet_data,k=10)

In [213]:
classifier2 = nltk.NaiveBayesClassifier.train(featuresets)
featuresets2 = [(conv_features(text,feature_words), party) for (text, party) in tweet_data_sample]

def estimator(index):
    est = classifier2.classify(featuresets2[index])
    return est

In [224]:
test_size = 3
test_set, train_set = featuresets2[:test_size], featuresets2[test_size:]
classifier2 = nltk.NaiveBayesClassifier.train(train_set)
classifier2.classify(nltk.tokenize('the country is in danger'))

TypeError: 'module' object is not callable

In [ ]:
for tweet, party in tweet_data_sample :
    estimated_party = ##
    
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [ ]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [ ]:
results

### Reflections

_Write a little about what you see in the results_ 